# Diamond Company Analytics Challenge

**Time:** 35 minutes

**Scenario:** You are the new data analyst at a prestigious diamond company. The CEO has called an urgent meeting and needs answers to critical business questions about the inventory.

**Your Mission:** Analyze the diamond inventory data and provide insights to guide strategic decisions.

In [ ]:
import polars as pl

# Load the diamonds dataset
print("Loading diamonds dataset...")
df = pl.read_csv('https://raw.githubusercontent.com/pycaret/pycaret/master/datasets/diamond.csv')
print("Dataset loaded successfully!")

Loading diamonds dataset...
Dataset loaded successfully!


## Section 1: Initial Inventory Assessment (5 minutes)

The CEO walks in: "Before we dive into specifics, I need to understand what we're working with. Give me the basics about our inventory database."

### Question 1: "How extensive is our inventory database?"

The CEO needs to know the scale of data you're analyzing.

In [ ]:
# Your code here:
print("Total records (diamonds):", df.shape[0])
print("Total attributes tracked:", df.shape[1])

Total records (diamonds): 6000
Total attributes tracked: 8


### Question 2: "What information do we track about each diamond?"

List all the characteristics we record.

In [ ]:
# Your code here:
print("Attributes tracked about each diamond:")
print(df.columns)

Attributes tracked about each diamond:
['Carat Weight', 'Cut', 'Color', 'Clarity', 'Polish', 'Symmetry', 'Report', 'Price']


### Question 3: "When I look at one line in this database, what am I looking at?"

Explain in business terms: Each of the row in the database represents on of the single diamond in the company inventory. In the database there is also the physical atrributes of the diamond, color, cut clarity and the price.

**Your answer:** Each row represents:

## Section 2: Strategic Business Questions (20 minutes)

The CEO continues: "Now let's get to the important questions. Our board meeting is tomorrow and I need concrete numbers."

### Question 4: "We're considering expanding our storage facilities. How many total carats do we have in inventory? I need to know if our vaults can handle the weight."

Think about what measurement would help answer this.

In [ ]:
# Your code here:

totals = df.select(
    pl.col("Carat Weight").cast(pl.Float64).fill_null(0).sum().alias("total_carats"),
    (pl.col("Carat Weight").cast(pl.Float64).fill_null(0).sum() * 0.2).alias("total_grams"),        # 1 carat = 0.2 grams
    (pl.col("Carat Weight").cast(pl.Float64).fill_null(0).sum() * 0.0002).alias("total_kilograms")  # grams -> kg
)

display(totals)

total_carats,total_grams,total_kilograms
f64,f64,f64
8007.12,1601.424,1.601424


### Question 5: "Our new marketing campaign claims we have 'the widest variety of cuts in the industry.' Can you verify this?"

The CEO wants to know about variety, not quantity.

In [ ]:
# Your code here:
unique_cuts = df.select(pl.col("Cut").unique())
num_cuts = df.select(pl.col("Cut").n_unique().alias("num_unique_cuts"))

print("Distinct cuts:")
display(unique_cuts)
print("\nNumber of unique cuts:")
display(num_cuts)

Distinct cuts:


Cut
str
"""Fair"""
"""Ideal"""
"""Signature-Ideal"""
"""Good"""
"""Very Good"""



Number of unique cuts:


num_unique_cuts
u32
5


### Question 6: "I'm meeting with our insurance company. They need to know our price exposure - what's our most and least valuable diamond?"

In [ ]:
# Your code here:
most_valuable = df.sort("Price", descending=True).head(1)
least_valuable = df.sort("Price").head(1)

print("Most valuable diamond:")
display(most_valuable)

print("\nLeast valuable diamond:")
display(least_valuable)

Most valuable diamond:


Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
f64,str,str,str,str,str,str,i64
2.79,"""Ideal""","""D""","""IF""","""EX""","""EX""","""GIA""",101561



Least valuable diamond:


Carat Weight,Cut,Color,Clarity,Polish,Symmetry,Report,Price
f64,str,str,str,str,str,str,i64
0.77,"""Good""","""I""","""VS1""","""VG""","""G""","""AGSL""",2184


### Question 7: "Which color grades should we focus on in our premium line? I need typical prices across our color spectrum."

The CEO wants to understand pricing patterns.

In [ ]:
# Your code here:
color_prices = df.group_by("Color").agg(
    pl.col("Price").mean().alias("avg_price"),
    pl.median("Price").alias("median_price"),
    pl.count().alias("num_diamonds")
).sort("avg_price", descending=True)

display(color_prices)

/tmp/ipython-input-112085331.py:5: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("num_diamonds")


Color,avg_price,median_price,num_diamonds
str,f64,f64,u32
"""D""",15255.783661,7567.0,661
"""F""",12712.241856,8860.0,1013
"""G""",12520.050633,8800.0,1501
"""E""",11539.190231,7059.5,778
"""H""",10487.347544,7169.0,1079
"""I""",8989.636364,6655.0,968


**Reflection:** After analyzing by color, how has your view of the data changed? What does each row in your result represent now?

[link text](https://)**Your answer:** New grain = After this anlysis I started seeing the data less as individual diamonds but more as a group result shwoing the overall trend.

## Section 3: Advanced Strategic Analysis (10 minutes)

The CEO leans forward: "These next questions will shape our strategy for next year."

### Question 8: "We're thinking of specializing in certain clarity grades. For our investor deck, I need a complete breakdown showing: how many diamonds we have in each clarity grade, their typical price, and the total carats we're holding for each grade."

In [ ]:
# Your code here:
clarity_breakdown = df.group_by("Clarity").agg([
    pl.count().alias("num_diamonds"),
    pl.mean("Price").alias("avg_price"),
    pl.median("Price").alias("median_price"),
    pl.sum("Carat Weight").alias("total_carats")
]).sort("avg_price", descending=True)

display(clarity_breakdown)

/tmp/ipython-input-550745019.py:3: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("num_diamonds"),


Clarity,num_diamonds,avg_price,median_price,total_carats
str,u32,f64,f64,f64
"""FL""",4,63776.0,62371.5,7.87
"""IF""",219,22105.844749,12647.0,316.0
"""VVS1""",285,16845.680702,11142.0,389.43
"""VVS2""",666,14142.177177,10266.0,876.58
"""VS1""",1192,13694.113255,9245.0,1682.74
"""VS2""",1575,11809.053333,7568.0,2170.81
"""SI1""",2059,8018.864012,5417.0,2563.69


### Question 9: "My luxury competitor claims they have the best prices for every cut-color combination. Show me our pricing structure across these combinations - I want to see where we're competitive."

In [ ]:
# Your code here:
pricing_by_cut_color = df.group_by(["Cut", "Color"]).agg(
    pl.col("Price").mean().alias("average_price"),
    pl.count().alias("num_diamonds")
).sort(["Cut", "Color"])

display(pricing_by_cut_color)

/tmp/ipython-input-985877042.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("num_diamonds")


Cut,Color,average_price,num_diamonds
str,str,f64,u32
"""Fair""","""D""",6058.25,12
"""Fair""","""E""",5370.625,32
"""Fair""","""F""",6063.625,24
"""Fair""","""G""",7345.52381,21
"""Fair""","""H""",5908.5,24
…,…,…,…
"""Very Good""","""E""",12101.910217,323
"""Very Good""","""F""",12413.905495,455
"""Very Good""","""G""",12354.013841,578


**Reflection:** How specific is your analysis now? What level of detail does each row represent?

**Your answer:** New grain = Very specific, shows the price by color number of diamonds and the average price. It's so much easier to understand the data now.

### Question 10: URGENT - "Our biggest client is interested in purchasing entire inventory segments. They asked: 'Which color-cut combinations represent your highest total inventory value?' Show me the top 5 most valuable inventory segments by total worth, and include how many diamonds are in each segment."

This is critical for tomorrow's negotiation.

In [ ]:
# Your code here:
top_segments = df.group_by(["Color", "Cut"]).agg([
    pl.sum("Price").alias("total_value"),
    pl.count().alias("num_diamonds")
]).sort("total_value", descending=True).head(5)

display(top_segments)

/tmp/ipython-input-23991976.py:4: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  pl.count().alias("num_diamonds")


Color,Cut,total_value,num_diamonds
str,str,i64,u32
"""G""","""Ideal""",9363514,690
"""G""","""Very Good""",7140620,578
"""F""","""Very Good""",5648327,455
"""F""","""Ideal""",5346782,363
"""H""","""Ideal""",5279687,458


## Debrief

Congratulations! You've just completed your first day as the diamond company's data analyst.

Remember:
- CEOs speak in business language, not technical terms
- Always understand what each row represents before analyzing
- Business questions contain clues about which operations to use